# DistilBERT IMDb Sentiment Fine-Tuning Recipe

DistilBERT compresses the knowledge of BERT into a lighter, faster model—perfect for production inference.  
This notebook documents **how we fine-tuned `distilbert-base-uncased` on the 50 k IMDb movie-review dataset for binary sentiment** and exported the resulting weights.

**Pipeline overview**

1. Load and stratify-split the IMDb dataset (90 % train / 10 % validation)  
2. Tokenise reviews with the DistilBERT tokenizer (`max_length = 256`)  
3. Train for three epochs with the Hugging Face *Trainer* API  
4. Evaluate (val accuracy ≈ 0.926, test accuracy ≈ 0.981)  
5. Save the best checkpoint to `distilbert-imdb/` for deployment

> *If you only need the trained model, download the release asset instead of re-running the heavy training step.*  



## Libraries

Import the core packages for data loading, tokenisation, model training, and evaluation.


In [ ]:
import torch, transformers, datasets, evaluate
print(f"torch        {torch.__version__}")
print(f"transformers {transformers.__version__}")
print(f"datasets     {datasets.__version__}")


## Load IMDb dataset


In [ ]:
from datasets import load_dataset


data_path = "imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv"

imdb_ds = load_dataset("csv", data_files={"train": data_path}, split="train")
print(imdb_ds)
print("\nSample ", imdb_ds[0])


In [ ]:
# 1. Split 90 % train / 10 % validation
split_ds = imdb_ds.train_test_split(test_size=0.1, seed=42)
train_ds = split_ds["train"]
val_ds   = split_ds["test"]

print("Train rows:", len(train_ds))
print("Val rows  :", len(val_ds))


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
label2id = {"negative": 0, "positive": 1}

def preprocess_batch(batch):
    batch["label"] = [label2id[s] for s in batch["sentiment"]]
    enc = tokenizer(batch["review"], padding="max_length",
                    truncation=True, max_length=256)
    batch.update(enc); return batch

tokenized_train = train_ds.map(preprocess_batch, batched=True,
                               remove_columns=["review", "sentiment"])
tokenized_val   =  val_ds.map(preprocess_batch, batched=True,
                               remove_columns=["review", "sentiment"])
tokenized_train.set_format("torch",
                           columns=["input_ids", "attention_mask", "label"])
tokenized_val.set_format("torch",
                         columns=["input_ids", "attention_mask", "label"])


In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import numpy as np
import evaluate

accuracy_metric = evaluate.load("accuracy")


#load pretrained model (num_labels=2 -> binary sentiment)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=preds, references=labels)

#training hyper-parameters
args = TrainingArguments(
    output_dir="checkpoints",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    seed=42,
    report_to=["none"]
)

#Trainer object
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)

#start fine-tuning (on Kaggle P100)
trainer.train()


In [ ]:
#Evaluate on the validation split
eval_metrics = trainer.evaluate()
print("Validation metrics ", eval_metrics)

#Confirm which checkpoint was judged 'best'
print("Best checkpoint path ", trainer.state.best_model_checkpoint)

#Save that best model for inference
trainer.save_model("distilbert-imdb")      # writes folder in /kaggle/working
print("Model saved to distilbert-imdb/")


In [ ]:
from transformers import AutoTokenizer

#load the same tokenizer you used for training
tok = AutoTokenizer.from_pretrained("distilbert-base-uncased")

#save it next to the model files
tok.save_pretrained("/kaggle/working/imdb-sentiment/distilbert-imdb")

#show the final contents
import os, glob, textwrap
files = glob.glob("/kaggle/working/imdb-sentiment/distilbert-imdb/*")
print(textwrap.fill('\n'.join(os.path.basename(f) for f in files), width=80))


In [ ]:

from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

#load model & tokenizer
MODEL_PATH = "imdb-sentiment/distilbert-imdb"   # relative to working dir
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model     = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
model.eval().to("cpu")                          # GPU not needed for demo

label_map = {0: "negative", 1: "positive"}

#FastAPI app 
app = FastAPI(title="IMDb Sentiment API")

class Item(BaseModel):
    text: str

@app.post("/predict")
def predict(item: Item):
    inputs = tokenizer(
        item.text,
        truncation=True,
        padding="max_length",
        max_length=256,
        return_tensors="pt",
    )
    with torch.no_grad():
        logits = model(**inputs).logits
        pred   = int(torch.argmax(logits, dim=-1))
        score  = float(torch.softmax(logits, dim=-1)[0, pred])
    return {"label": label_map[pred], "confidence": round(score, 4)}



In [ ]:
from fastapi.testclient import TestClient
import importlib.util, sys, pathlib

# Dynamically import the api module we just wrote
spec = importlib.util.spec_from_file_location(
    "imdb_api", pathlib.Path("/kaggle/working/imdb-sentiment/api/main.py")
)
api_module = importlib.util.module_from_spec(spec)
sys.modules["imdb_api"] = api_module
spec.loader.exec_module(api_module)

client = TestClient(api_module.app)

resp = client.post("/predict", json={"text": "A surprisingly fun movie!"})
print("Status code:", resp.status_code)
print("Response JSON:", resp.json())


In [ ]:
from datasets import load_dataset
import numpy as np, torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_PATH = "imdb-sentiment/distilbert-imdb"

# 1. Load test split (25 000 reviews)
test_ds = load_dataset("imdb", split="test")

# 2. Load tokenizer & model
tok   = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to("cuda").eval()

# 3. Batched inference
batch_size = 64
correct = 0
for i in range(0, len(test_ds), batch_size):
    texts  = test_ds[i : i + batch_size]["text"]
    labels = test_ds[i : i + batch_size]["label"]
    enc    = tok(texts, padding=True, truncation=True, max_length=256, return_tensors="pt").to("cuda")
    with torch.no_grad():
        preds = torch.argmax(model(**enc).logits, dim=-1).cpu().numpy()
    correct += int(np.sum(preds == labels))

test_acc = correct / len(test_ds)
print(f"✅ Test accuracy: {test_acc:.4f}")
